In [6]:
import sys
sys.path.append('..')

In [7]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI
from langchain_experimental.llms.ollama_functions import OllamaFunctions
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0
)

In [8]:
from tools.logger import getLogger
logging = getLogger()

In [9]:

# from langchain_community.document_loaders import PyPDFLoader
from typing import List
from langchain_core.documents import Document
from tools.PDFTablePyPlumberLoader import PDFTablePyPlumberLoader
pages: List[List[Document]] = []
dir = '../data'
for filename in os.listdir(dir):
    if filename.endswith(".pdf"):
        if '個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版' not in filename:
            continue
        loader = PDFTablePyPlumberLoader(os.path.join(dir, filename), llm)
        pages += [loader.load()]
pages

Processing documents: 100%|██████████| 32/32 [00:36<00:00,  1.15s/it]


[[Document(metadata={'source': '../data/個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf', 'page_number': 1}, page_content='資安等級：內部\n台灣人壽\n個人保險首/續期繳費暨保單貸款還款息作業手冊\n(113 年 7 月版)_業務通路版'),
  Document(metadata={'source': '../data/個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf', 'page_number': 2}, page_content='\n\n1. 配合實務作業，調整續期保險費採自行繳費作業之繳費管道說明。\n\n### 繳費管道變更與繳費日認定原則\n\n#### 原指定繳費管道：金融機構\n\n1. **改自行匯款**\n   - **於一照期限內完成**：是，於一照期限內完成匯款\n   - **繳費日認定原則**：可以用原授權書之申請日為繳費日\n   - **未於一照期限內完成**：否，未於一照期限內完成匯款\n   - **繳費日認定原則**：以實際匯款日為繳費日\n\n2. **改他行金融機構轉帳**\n   - **完成**：不限\n   - **繳費日認定原則**：可以用原授權書之申請日為繳費日\n\n3. **改信用卡**\n   - **完成**：不限\n   - **繳費日認定原則**：以新信用卡授權書之申請日為繳費日\n\n#### 原指定繳費管道：信用卡\n\n1. **改自行匯款**\n   - **完成**：不限\n   - **繳費日認定原則**：以實際匯款日為繳費日\n\n2. **改金融機構轉帳**\n   - **完成**：不限\n   - **繳費日認定原則**：以新金融機構轉帳授權書之申請日為繳費日\n\n3. **改同銀行其他信用卡或改他行信用卡**\n   - **完成**：不限\n   - **繳費日認定原則**：可以用原信用卡授權書之申請日為繳費日\n\n#### 註釋\n\n- **註1**：一照，意指核印/扣款失敗的第一次照會通知，期限為14天，或案件考量時效個別調整為7天者。\n- **註2**：停售商品變更繳費管道後之繳費日若等於或晚於商品停售日，應予以退件。\n

In [10]:
'''將表格的描述加入到該頁結尾'''
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import traceback

class QuestionAnswer(BaseModel):
    question: str = Field(
        description="客戶提出的問題"
    )
    answer: str = Field(
        description="相對應的答案"
    )
    
class QuestionAnswerPack(BaseModel):
    question_answers: Optional[List[QuestionAnswer]] = Field(
        description="客戶可能提出的問題與答案清單"
    )
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
"""你是一個客戶服務問答專家，根據提供的資訊，提供客戶可能提的問題，以及相對應的答案，
如果提供的資訊不足以產生問答，不要捏造任何未提供的資訊，你可以縮小問答數量來提供正確的資訊
"""
    ),
    (
        "human",
        """請根據參考內容，模擬客戶可能會問的問題與答案，提供 {question_num} 個問答\n\n提供的資訊:\n{ref_str}""",
    ),
])
desc_llm = llm.with_structured_output(
    QuestionAnswerPack
)
desc_chain = prompt | desc_llm

def qa_generator(question_num: int, ref_str: str):
    try:
        if question_num > 50:
            question_num = 50
        if question_num < 1:
            question_num = 1
        qas = desc_chain.invoke({"ref_str": ref_str, "question_num": question_num}).question_answers
        if not qas:
            return []
        return qas
    except Exception as e:
        logging.error(e)
        logging.error(traceback.format_exc())
        return []
        
    

In [11]:
qas = qa_generator(10, pages[0][3])
qas

[QuestionAnswer(question='保險業務員可以代收保戶的保險費嗎？', answer='根據規定，保險業務員不得代收保戶的保險費，包括現金、支票等形式的繳納。'),
 QuestionAnswer(question='新契約的保險什麼時候生效？', answer='新契約投保於核保通過且繳交首期保險費後，保險才會生效。'),
 QuestionAnswer(question='續期保險費的繳納方式可以變更嗎？', answer='續期繳費方式可以隨時申請變更，但需在應繳費日前20天申請並檢附相關授權書。'),
 QuestionAnswer(question='有哪些繳費方式？', answer='繳費方式有躉繳、分期繳（月繳、季繳、半年繳、年繳）和彈性繳（適用於超額保險費）。'),
 QuestionAnswer(question='月繳件首期應繳交多少保險費？', answer='月繳件首期應繳交2個月的保險費，除非商品或特定通路另有規定。'),
 QuestionAnswer(question='保單生效日期可以溯前嗎？', answer='根據主管機關函令，保單生效日期不得溯前。'),
 QuestionAnswer(question='傳統型商品的繳別因子是多少？', answer='傳統型商品的繳別因子為：半年繳0.520，季繳0.262，月繳0.088。'),
 QuestionAnswer(question='投資型商品的繳別因子需要換算嗎？', answer='投資型商品的目標保險採分期繳納者，無須依繳別係數換算。'),
 QuestionAnswer(question='附約的繳費方式與主約相同嗎？', answer='原則上，附約的繳費方式與主約相同，但若商品另有規定則除外。'),
 QuestionAnswer(question='信用卡扣款需要什麼手續？', answer='信用卡扣款作業須完成信用卡身份驗證機制，並需檢附「保險費信用卡付款授權書」。')]

In [31]:
from typing import Dict
from concurrent.futures import Future

qa_num = 10
futures = []
future_map: Dict[Future, Document] = {}

qas: List[QuestionAnswer] = []
with ThreadPoolExecutor(max_workers=5) as executor:
    for doc_pages in pages:
        for page in doc_pages:
            future = executor.submit(qa_generator, qa_num, page.page_content)
            futures.append(future)
            future_map[future] = page
    for future in tqdm(
        as_completed(futures), total=len(future_map), desc="Processing documents"
    ):
        try:
            qas += future.result()
        except Exception as e:
            logging.error(f"error: {e}, page_content: {future_map[future].page_content}")
            logging.error(traceback.format_exc())

什麼是保險？
保險是一種風險管理工具，通過支付保費，您可以在發生特定事件時獲得經濟補償。
----------------------------------------
我需要什麼類型的保險？
這取決於您的個人情況和需求。常見的保險類型包括健康保險、人壽保險、汽車保險和房屋保險。
----------------------------------------
保險費用如何計算？
保險費用根據多種因素計算，包括您的年齡、健康狀況、保險類型和保額等。
----------------------------------------
如何申請保險理賠？
您需要聯繫您的保險公司，提供相關文件和證據，如醫療報告、警察報告等，然後按照指示完成理賠申請。
----------------------------------------
保險的等待期是什麼？
等待期是指從保單生效到可以提出理賠申請的時間段。在此期間內發生的事件通常不在保險範圍內。
----------------------------------------
什麼是免賠額？
免賠額是指在保險公司開始支付理賠之前，您需要自行承擔的費用。
----------------------------------------
保險的保額是什麼意思？
保額是指保險公司在發生保險事故時，根據保單條款所支付的最高金額。
----------------------------------------
我可以取消保險嗎？
是的，您可以隨時取消保險，但可能會有一些手續費或損失部分已支付的保費。
----------------------------------------
保險公司如何評估風險？
保險公司通過分析多種因素來評估風險，包括您的年齡、健康狀況、職業、生活習慣等。
----------------------------------------
什麼是保險的除外責任？
除外責任是指保單中明確列出的不在保險範圍內的情況或事件。
----------------------------------------
